In [1]:
#| hide
from nbdev.showdoc import *

# KNN Label Transfer

## The `KNN_Label_transfer` module provides functionality for label transfer between datasets using k-nearest neighbors (KNN) algorithms. It includes methods for majority and weighted voting based on nearest neighbors, calculation of centroids for labeled data, and label assignment based on nearest centroids. This module is designed to facilitate the propagation of labels from a reference dataset with known labels to a query dataset where labels are unknown.


In [2]:
#| default_exp KNN_Label_transfer


In [3]:
#| export
def knn_majority_voting(indices, reference_labels):
    "Determines the final label for each point in the query dataset using majority voting based on the labels of the k nearest neighbors in the reference dataset."
    query_labels = []
    for ind in indices:
        neighbor_labels = [reference_labels[i] for i in ind]
        label_counts = Counter(neighbor_labels)
        most_common_label = label_counts.most_common(1)[0][0]
        query_labels.append(most_common_label)
    return query_labels


In [4]:
#| export
def knn_weighted_voting(indices, distances, reference_labels):
    "Determines the final label for each point in the query dataset using weighted voting based on the labels and distances of the k nearest neighbors in the reference dataset."
    query_labels = []
    for ind, dist in zip(indices, distances):
        weighted_votes = {}
        for i, d in zip(ind, dist):
            label = reference_labels[i]
            weight = 1 / (d + 1e-6)  # Adding a small constant to avoid division by zero
            weighted_votes[label] = weighted_votes.get(label, 0) + weight
        most_common_label = max(weighted_votes, key=weighted_votes.get)
        query_labels.append(most_common_label)
    return query_labels


In [5]:
#| export
def calculate_centroids(reference_data, reference_labels):
    "Calculates the centroids for each label in the reference dataset."
    label_sums = defaultdict(lambda: np.zeros(len(reference_data[0])))
    label_counts = Counter(reference_labels)
    for data, label in zip(reference_data, reference_labels):
        label_sums[label] += np.array(data)
    centroids = {label: label_sums[label] / count for label, count in label_counts.items()}
    return centroids


In [6]:
#| export
def assign_labels_by_nearest_centroid(query_data, centroids):
    "Assigns labels to each point in the query dataset based on the nearest centroid."
    query_labels = []
    for data in query_data:
        data_point = np.array(data)
        closest_label = min(centroids.keys(), key=lambda label: np.linalg.norm(data_point - centroids[label]))
        query_labels.append(closest_label)
    return query_labels

In [7]:
#| export
def run_label_transfer(embedding_array_reference, embedding_array_query, reference_labels,
                       k=1, use_gpu=True, batch_size=None, distance_metric='L2',
                       label_consensus='majority_voting', timed=False):
    "Transfers labels from a reference dataset to a query dataset using FAISS."
    
    
    from collections import Counter, defaultdict
    import numpy as np
    import faiss
    import time
    
    start_time = time.time()
    dimension = embedding_array_reference.shape[1]
    res = faiss.StandardGpuResources() if use_gpu else None
    if use_gpu:
        res.noTempMemory()
    if distance_metric == 'L2':
        index = faiss.GpuIndexFlatL2(res, dimension) if use_gpu else faiss.IndexFlatL2(dimension)
    elif distance_metric == 'IP':
        index = faiss.GpuIndexFlatIP(res, dimension) if use_gpu else faiss.IndexFlatIP(dimension)
    else:
        raise ValueError("Invalid distance metric. Choose 'L2' or 'IP'.")
    index.add(embedding_array_reference)
    num_query_points = embedding_array_query.shape[0]
    batch_size = batch_size or num_query_points
    query_labels = []
    for i in range(0, num_query_points, batch_size):
        batch_query = embedding_array_query[i:i + batch_size]
        distances, indices = index.search(batch_query, k)
        if k > 1 and label_consensus == 'majority_voting':
            batch_labels = knn_majority_voting(indices, reference_labels)
        elif k > 1 and label_consensus == 'weighted_voting':
            batch_labels = knn_weighted_voting(indices, distances, reference_labels)
        elif label_consensus == 'centroid_based':
            if i == 0:  # Calculate centroids only once
                centroids = calculate_centroids(embedding_array_reference, reference_labels)
            batch_labels = assign_labels_by_nearest_centroid(batch_query, centroids)
        else:
            batch_labels = [reference_labels[i[0]] for i in indices]
        query_labels.extend(batch_labels)
    end_time = time.time()
    duration_minutes = (end_time - start_time) / 60
    if timed:
        return query_labels, duration_minutes
    else:
        return query_labels
# End of Selection

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()

